# Day 13: Packet Scanners

You need to cross a vast firewall. The firewall consists of several layers, each with a security scanner that moves back and forth across the layer. To succeed, you must not be detected by a scanner.

By studying the firewall briefly, you are able to record (in your puzzle input) the depth of each layer and the range of the scanning area for the scanner within it, written as depth: range. Each layer has a thickness of exactly 1. A layer at depth 0 begins immediately inside the firewall; a layer at depth 1 would start immediately after that.

For example, suppose you've recorded the following:

    0: 3
    1: 2
    4: 4
    6: 4

This means that there is a layer immediately inside the firewall (with range 3), a second layer immediately after that (with range 2), a third layer which begins at depth 4 (with range 4), and a fourth layer which begins at depth 6 (also with range 4). Visually, it might look like this:

     0   1   2   3   4   5   6
    [ ] [ ] ... ... [ ] ... [ ]
    [ ] [ ]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

Within each layer, a security scanner moves back and forth within its range. Each security scanner starts at the top and moves down until it reaches the bottom, then moves up until it reaches the top, and repeats. A security scanner takes one picosecond to move one step. Drawing scanners as S, the first few picoseconds look like this:

    Picosecond 0:
     0   1   2   3   4   5   6
    [S] [S] ... ... [S] ... [S]
    [ ] [ ]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

    Picosecond 1:
     0   1   2   3   4   5   6
    [ ] [ ] ... ... [ ] ... [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

    Picosecond 2:
     0   1   2   3   4   5   6
    [ ] [S] ... ... [ ] ... [ ]
    [ ] [ ]         [ ]     [ ]
    [S]             [S]     [S]
                    [ ]     [ ]

    Picosecond 3:
     0   1   2   3   4   5   6
    [ ] [ ] ... ... [ ] ... [ ]
    [S] [S]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [S]     [S]

Your plan is to hitch a ride on a packet about to move through the firewall. The packet will travel along the top of each layer, and it moves at one layer per picosecond. Each picosecond, the packet moves one layer forward (its first move takes it into layer 0), and then the scanners move one step. If there is a scanner at the top of the layer as your packet enters it, you are caught. (If a scanner moves into the top of its layer while you are there, you are not caught: it doesn't have time to notice you before you leave.) If you were to do this in the configuration above, marking your current position with parentheses, your passage through the firewall would look like this:

Initial state:

     0   1   2   3   4   5   6
    [S] [S] ... ... [S] ... [S]
    [ ] [ ]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

    Picosecond 0:
     0   1   2   3   4   5   6
    (S) [S] ... ... [S] ... [S]
    [ ] [ ]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

     0   1   2   3   4   5   6
    ( ) [ ] ... ... [ ] ... [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]


    Picosecond 1:
     0   1   2   3   4   5   6
    [ ] ( ) ... ... [ ] ... [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

     0   1   2   3   4   5   6
    [ ] (S) ... ... [ ] ... [ ]
    [ ] [ ]         [ ]     [ ]
    [S]             [S]     [S]
                    [ ]     [ ]


    Picosecond 2:
     0   1   2   3   4   5   6
    [ ] [S] (.) ... [ ] ... [ ]
    [ ] [ ]         [ ]     [ ]
    [S]             [S]     [S]
                    [ ]     [ ]

     0   1   2   3   4   5   6
    [ ] [ ] (.) ... [ ] ... [ ]
    [S] [S]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [S]     [S]


    Picosecond 3:
     0   1   2   3   4   5   6
    [ ] [ ] ... (.) [ ] ... [ ]
    [S] [S]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [S]     [S]

     0   1   2   3   4   5   6
    [S] [S] ... (.) [ ] ... [ ]
    [ ] [ ]         [ ]     [ ]
    [ ]             [S]     [S]
                    [ ]     [ ]


    Picosecond 4:
     0   1   2   3   4   5   6
    [S] [S] ... ... ( ) ... [ ]
    [ ] [ ]         [ ]     [ ]
    [ ]             [S]     [S]
                    [ ]     [ ]

     0   1   2   3   4   5   6
    [ ] [ ] ... ... ( ) ... [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]


    Picosecond 5:
     0   1   2   3   4   5   6
    [ ] [ ] ... ... [ ] (.) [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

     0   1   2   3   4   5   6
    [ ] [S] ... ... [S] (.) [S]
    [ ] [ ]         [ ]     [ ]
    [S]             [ ]     [ ]
                    [ ]     [ ]


    Picosecond 6:
     0   1   2   3   4   5   6
    [ ] [S] ... ... [S] ... (S)
    [ ] [ ]         [ ]     [ ]
    [S]             [ ]     [ ]
                    [ ]     [ ]

     0   1   2   3   4   5   6
    [ ] [ ] ... ... [ ] ... ( )
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

In this situation, you are caught in layers 0 and 6, because your packet entered the layer when its scanner was at the top when you entered it. You are not caught in layer 1, since the scanner moved into the top of the layer once you were already there.

The severity of getting caught on a layer is equal to its depth multiplied by its range. (Ignore layers in which you do not get caught.) The severity of the whole trip is the sum of these values. In the example above, the trip severity is 0*3 + 6*4 = 24.

Given the details of the firewall you've recorded, if you leave immediately, what is the severity of your whole trip?

In [1]:
from collections import defaultdict
from operator import add, sub

class Scanner:
    def __init__(self, fwrange):
        self.fwrange = fwrange
        self.start_state = 1
        self.reset()
    
    def advance(self):
        if self.state == self.fwrange:
            self.direction = sub
        if self.state == 1:
            self.direction = add
        
        self.state = self.direction(self.state, 1)
        return self
    
    def reset(self):
        self.state = self.start_state
        self.direction = add
        
    def __repr__(self):
        return "Scanner({})".format(self.fwrange)
    
    def __str__(self):
        parts = []
        for i in range(1, self.fwrange + 1):
            if i == self.state:
                parts.append('[S]')
            else:
                parts.append('[ ]')
        return " ".join(parts)

class EmptyScanner(Scanner):
    def __init__(self):
        self.fwrange = -1
        self.start_state = -1
        self.reset()
        
    def advance(self):
        return self
    
    def __repr__(self):
        return "EmptyScanner()"
    
    def __str__(self):
        return '…'
    
def construct_firewall(lines):
    fw = defaultdict(EmptyScanner)
    
    for line in lines:
        depth, fwrange = line.strip().split(': ')
        depth, fwrange = int(depth), int(fwrange)
        fw[depth] = Scanner(fwrange)
    
    return [fw[i] for i in range(max(fw.keys()) + 1)]

def follow_packet(firewall, trace=False):
    severity = 0
    
    for step, layer in enumerate(firewall):
        if trace:
            print('Start of step', step)

        if layer.state == 1:
            if trace:
                print("Caught by scanner")
            severity += (layer.fwrange * step)
            
        for scanner in firewall:
            if trace:
                print(scanner)
            scanner.advance()
        
        if trace:
            print('------------')
    
    return severity

In [2]:
testcase = """0: 3
1: 2
4: 4
6: 4""".split('\n')

fw = construct_firewall(testcase)
assert follow_packet(fw) == 24

In [3]:
challenge = open('input/day13.txt').readlines()
fw = construct_firewall(challenge)
follow_packet(fw)

1876

## Part Two

Now, you need to pass through the firewall without being caught - easier said than done.

You can't control the speed of the packet, but you can delay it any number of picoseconds. For each picosecond you delay the packet before beginning your trip, all security scanners move one step. You're not in the firewall during this time; you don't enter layer 0 until you stop delaying the packet.

In the example above, if you delay 10 picoseconds (picoseconds 0 - 9), you won't get caught:

    State after delaying:
     0   1   2   3   4   5   6
    [ ] [S] ... ... [ ] ... [ ]
    [ ] [ ]         [ ]     [ ]
    [S]             [S]     [S]
                    [ ]     [ ]
    
    Picosecond 10:
     0   1   2   3   4   5   6
    ( ) [S] ... ... [ ] ... [ ]
    [ ] [ ]         [ ]     [ ]
    [S]             [S]     [S]
                    [ ]     [ ]
    
     0   1   2   3   4   5   6
    ( ) [ ] ... ... [ ] ... [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]
    
    
    Picosecond 11:
     0   1   2   3   4   5   6
    [ ] ( ) ... ... [ ] ... [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]
    
     0   1   2   3   4   5   6
    [S] (S) ... ... [S] ... [S]
    [ ] [ ]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [ ]     [ ]
    
    
    Picosecond 12:
     0   1   2   3   4   5   6
    [S] [S] (.) ... [S] ... [S]
    [ ] [ ]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [ ]     [ ]
    
     0   1   2   3   4   5   6
    [ ] [ ] (.) ... [ ] ... [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]
    
    
    Picosecond 13:
     0   1   2   3   4   5   6
    [ ] [ ] ... (.) [ ] ... [ ]
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]
    
     0   1   2   3   4   5   6
    [ ] [S] ... (.) [ ] ... [ ]
    [ ] [ ]         [ ]     [ ]
    [S]             [S]     [S]
                    [ ]     [ ]
    
    
    Picosecond 14:
     0   1   2   3   4   5   6
    [ ] [S] ... ... ( ) ... [ ]
    [ ] [ ]         [ ]     [ ]
    [S]             [S]     [S]
                    [ ]     [ ]

     0   1   2   3   4   5   6
    [ ] [ ] ... ... ( ) ... [ ]
    [S] [S]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [S]     [S]
    
    
    Picosecond 15:
     0   1   2   3   4   5   6
    [ ] [ ] ... ... [ ] (.) [ ]
    [S] [S]         [ ]     [ ]
    [ ]             [ ]     [ ]
                    [S]     [S]

     0   1   2   3   4   5   6
    [S] [S] ... ... [ ] (.) [ ]
    [ ] [ ]         [ ]     [ ]
    [ ]             [S]     [S]
                    [ ]     [ ]
    
    
    Picosecond 16:
     0   1   2   3   4   5   6
    [S] [S] ... ... [ ] ... ( )
    [ ] [ ]         [ ]     [ ]
    [ ]             [S]     [S]
                    [ ]     [ ]

     0   1   2   3   4   5   6
    [ ] [ ] ... ... [ ] ... ( )
    [S] [S]         [S]     [S]
    [ ]             [ ]     [ ]
                    [ ]     [ ]

Because all smaller delays would get you caught, the fewest number of picoseconds you would need to delay to get through safely is 10.

What is the fewest number of picoseconds that you need to delay the packet to pass through the firewall without being caught?

In [4]:
class Firewall:
    def __init__(self, lines, trace=False):
        self.trace = trace
        self.caught_packet = False
        fw = defaultdict(EmptyScanner)
    
        for line in lines:
            depth, fwrange = line.strip().split(': ')
            depth, fwrange = int(depth), int(fwrange)
            fw[depth] = Scanner(fwrange)
    
        self.state = [fw[i] for i in range(max(fw.keys()) + 1)]
    
    def advance(self):
        for scanner in self.state:
            if self.trace:
                print(scanner)
            scanner.advance()

    def reset(self):
        self.caught_packet = False
        for scanner in self.state:
            scanner.reset()
            
    def traverse(self):
        severity = 0
    
        for step, layer in enumerate(self.state):
            if self.trace:
                print('Start of step', step)

            if layer.state == 1:
                if self.trace:
                    print("Caught by scanner")
                self.caught_packet = True
                severity += (layer.fwrange * step)
            
            self.advance()
            
            if self.trace:
                print('------------')
    
        return severity
    
def break_firewall(fw):
    broken = False
    delay = 0
    
    while not broken:
        fw.reset()
        
        for i in range(delay):
            fw.advance()
            
        severity = fw.traverse()
        
        if not fw.caught_packet:
            broken = True
        else:
            delay += 1
            
    return delay

In [5]:
# TEST
fw = Firewall(testcase)
assert break_firewall(fw) == 10

The method used above works very poorly on the challenge input, as it's *very* inefficient. Let's approach it a simpler way:

In [6]:
from itertools import count

def create_firewall(lines):
    fw = defaultdict(int)

    for line in lines:
        layer, _range = map(int, line.strip().split(': '))
        fw[layer] = _range * 2 - 2

    return [fw[i] for i in range(max(fw) + 1)]


def follow_packet(fw, delay=0, debug=False):
    severity = 0
    caught = False

    start = delay
    stop = start + len(fw)

    for step, layer in zip(range(start, stop), fw):
        if layer == 0:
            continue

        if step % layer == 0:
            if debug:
                debugmsg = 'Caught at {step} (depth of layer: {depth})'
                print(debugmsg.format(step=step, depth=layer))
            caught = True
            severity += step * ((layer + 2) // 2)

    return (caught, severity)

Then we test the above code on the test case, to make sure it yields the same result:

In [7]:
test_fw = create_firewall(testcase)
print(follow_packet(test_fw, debug=True))
assert follow_packet(test_fw)[1] == 24

Caught at 0 (depth of layer: 4)
Caught at 6 (depth of layer: 6)
(True, 24)


… and then we test the new code against the previous solution for part 1:

In [8]:
fw = create_firewall(challenge)
assert follow_packet(fw)[1] == 1876

Finally, we add code to find out when to enter the firewall:

In [9]:
def break_firewall(fw, debug=False):
    for delay in count():
        if debug and delay % 1000000 == 0:
            print('delay =', delay)
        stopped, severity = follow_packet(fw, delay)
        if not stopped:
            return delay

In [10]:
break_firewall(fw)

3964778

## Timings

Some timing of the previous functions:

In [11]:
from timeit import timeit

result = timeit('break_firewall(fw)',
                number=10,
                setup='from __main__ import break_firewall, fw')
print('Timing for break_firewall(fw) =', result)

Timing for break_firewall(fw) = 313.7110795590015
